See readme.md for ideal fields and descriptions

In [21]:
# ! pip install pandas pyarrow
# ! pip install polars
# ! pip install maturin
# ! pip install cryo

In [22]:
import subprocess
import os
from datetime import datetime, timedelta, timezone
import time
import pyarrow.parquet as pq
import pandas as pd
import cryo
import sys
sys.path.append("../../helper_functions")
import web3py_utils as w3py
sys.path.pop()
# test adding cryo_cli python

'../../helper_functions'

In [23]:
# import polars as pl

In [24]:
# Test doing Degen Chain
rpc_url = 'https://rpc.degen.tips'
chain_name = 'degen'

# Lyra
# rpc_url = 'https://rpc.lyra.finance/'
# chain_name = 'lyra'

block_time_sec = 2
###
trailing_days = 0.25
dry_run = False
fields = ['blocks', 'txs']
requests_per_second_max = 500 # -1 means ignore


In [25]:
#Intermediate Calc
blocks_per_day = (60*60*24) / block_time_sec
# print(blocks_per_day)

In [26]:
# Init timestamps


current_date_utc = datetime.utcnow().date()
# Convert the current date to a Unix timestamp (remaining in UTC)
current_timestamp_utc = datetime(current_date_utc.year, current_date_utc.month, current_date_utc.day, tzinfo=timezone.utc).timestamp()
previous_date_utc = current_date_utc - timedelta(days=trailing_days)
previous_timestamp_utc = datetime(previous_date_utc.year, previous_date_utc.month, previous_date_utc.day, tzinfo=timezone.utc).timestamp()

current_block = w3py.getLatestBlockNumber(rpc_url)
previous_block = current_block - (trailing_days * blocks_per_day)

print('end: ' + str(previous_block))
print('start: ' + str(current_block))

end: 7256567.0
start: 7267367


In [27]:

start_timestamp = int(previous_timestamp_utc)
end_timestamp = int(current_timestamp_utc)

start_time = time.time()

In [28]:
# Generate Command
# if dry_run == 1:
#     dry_txt = '--dry'
# else:
#     dry_txt = ''
# if requests_per_second_max > -1:
#     rps_txt = '--requests-per-second ' + str(requests_per_second_max)
# else:
#     rps_txt = ''

# Fetch and save blocks data in JSON
data = cryo.freeze(
    ['txs', 'blocks'],
    blocks=[str(previous_block) + ":" + str(current_block)],
    rpc=rpc_url,
    output_dir="outputs/",
    file_format='parquet',
    label=chain_name,
    hex=True,
    dry=dry_run,
    requests_per_second=requests_per_second_max
)
# command = f"cryo {fields} --rpc {rpc_url} --timestamps {start_timestamp}:{end_timestamp} --subdirs datatype --label {chain_name} {rps_txt} {dry_txt}"
# print(command)

cryo parameters
───────────────
- version: 152a635
- data: 
    - datatypes: blocks, transactions
    - blocks: n=10,800 min=7,256,567 max=7,267,366 align=no reorg_buffer=0
- source: 
    - network: network_666666666
    - rpc url: https://rpc.degen.tips
    - max requests per second: 500
    - max concurrent requests: 500
    - max concurrent chunks: 4
- output: 
    - chunk size: 1,000
    - chunks to collect: 11 / 11
    - output format: parquet
    - output dir: /Users/michaelsilberling/Documents/GitHub/op-analytics/op_chains_tracking/lightweight_chain_tracker/outputs
    - report file: $OUTPUT_DIR/.cryo/reports/2024-04-18_11-27-56.235469.json


schema for blocks
─────────────────
- block_number: uint32
- block_hash: hex
- timestamp: uint32
- author: hex
- gas_used: uint64
- extra_data: hex
- base_fee_per_gas: uint64
- chain_id: uint64

sorting blocks by: block_number

other available columns: parent_hash, state_root, transactions_root, receipts_root, logs_bloom, total_difficulty, 

In [29]:

# Run the command using subprocess.run and capture the output
# result = subprocess.run(
#     command, 
#     shell=True, 
#     stdout=subprocess.PIPE,  # Capture standard output
#     stderr=subprocess.PIPE,  # Capture standard error
#     text=True  # Capture output as text (Python 3.7+)
# )

# # Display the captured output
# if result.returncode == 0:
#     print("Command succeeded. Output:")
#     print(result.stdout)
# else:
#     print("Command failed. Error output:")
#     print(result.stderr)

end_time = time.time()

In [30]:
# Calculate the elapsed time
elapsed_time = end_time - start_time
# Print the elapsed time in seconds
print(f"Elapsed time: {elapsed_time:.4f} seconds")

Elapsed time: 0.1521 seconds


In [ ]:
# # Read parquet files
txs = pl.scan_parquet('transactions__' + chain_name + '/*.parquet')
blocks = pl.scan_parquet('blocks__' + chain_name + '/*.parquet')

# Rename the 'gas_used' column to 'block_gas_used' in the 'blocks' DataFrame
blocks = blocks.rename({"gas_used": "block_gas_used"})

# Perform the join on 'block_number' and 'chain_id'
joined_df = blocks.join(
    txs,
    on=["block_number", "chain_id"],
    how="inner"  # You can specify the type of join you want (inner, outer, left, right)
)

# Convert Unix timestamp to datetime and create a new column 'timestamp_dt'
joined_df = joined_df.with_columns(
    pl.from_epoch("timestamp", time_unit="s").alias("timestamp_dt")
)

# Truncate the 'timestamp_dt' column to the day and create a new column 'timestamp_date'
joined_df = joined_df.with_columns(
    pl.col("timestamp_dt").dt.truncate("1d").alias("timestamp_date")
)

In [ ]:
# print(blocks.schema)
# print(txs.schema)
print(joined_df.schema)

#test output
joined_pd = joined_df.collect().to_pandas()
joined_pd.tail(5)

print('num blocks: ' + str(joined_pd['block_number'].nunique()))

In [ ]:
# Create a reference list of consecutive block numbers
reference_block_numbers = list(range(joined_pd['block_number'].min(), joined_pd['block_number'].max() + 1))

# Find the missing block numbers
missing_block_numbers = set(reference_block_numbers) - set(joined_pd['block_number'])

print("Missing block numbers:", missing_block_numbers)

# Convert the missing_block_numbers set to a list and sort it
missing_block_numbers_list = sorted(list(missing_block_numbers))
is_consecutive = all(missing_block_numbers_list[i] == missing_block_numbers_list[i - 1] + 1 for i in range(1, len(missing_block_numbers_list)))

if is_consecutive:
    print("The missing block numbers are consecutive.")
else:
    print("The missing block numbers are not consecutive.")

In [ ]:
# Assuming you have a DataFrame named 'joined_df' with the required columns

result_df = joined_df.group_by([pl.col("timestamp_date"), pl.col("chain_id")]).agg(
    num_blocks=pl.col("block_number").n_unique(),
    min_block_number=pl.col("block_number").min(),
    max_block_number=pl.col("block_number").max(),
    min_block_time=pl.col("timestamp").min(),
    max_block_time=pl.col("timestamp").max(),

    num_user_transactions=
        pl.when(pl.col("gas_price") > 0).then(pl.col("transaction_hash")).count(),
    num_success_user_transactions=
        pl.when((pl.col("gas_price") > 0) & pl.col("success")).then(pl.col("transaction_hash")).count(),
    num_senders=pl.col("from_address").filter(pl.col("gas_price") > 0).n_unique(),

    total_gas_used=pl.col("gas_used").sum(),
    user_gas_used=pl.col("gas_used").filter(pl.col("gas_price") > 0).sum(),
    total_gas_used_per_block = pl.col("gas_used").sum() / pl.col("block_number").n_unique(),
    user_gas_used_per_block = pl.col("gas_used").filter(pl.col("gas_price") > 0).sum() / pl.col("block_number").n_unique(),
    
    l2_fees_base_fees_eth=(pl.col("base_fee_per_gas") * pl.col("gas_used")).sum() / 1e18,
    l2_fees_priority_fees_eth=pl.when(pl.col("gas_price") > 0).then((pl.col("gas_price") - pl.col("base_fee_per_gas")) * pl.col("gas_used")).sum() / 1e18,
    l2_fees_total_fees_eth=(pl.col("gas_price") * pl.col("gas_used")).sum() / 1e18,
)
result_df

In [ ]:
#Execute and turn to pandas
result_df = result_df.collect().to_pandas()


In [ ]:
# Filter
result_df = result_df[result_df['min_block_time']>= start_timestamp] #seems like 1 block before gets pulled. yolo.

In [ ]:

result_df['min_block_time_dt'] = pd.to_datetime(result_df['min_block_time'], unit='s')
result_df['max_block_time_dt'] = pd.to_datetime(result_df['max_block_time'], unit='s')
display(result_df.sort_values(by='timestamp_date',ascending=False))